In [2]:
import os
import PyPDF2
import re
import subprocess
import tempfile

In [3]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# ✅ Example usage
pdf_path = "Academic_Papers/Attention_is_all_you_need.pdf"
text = extract_text_from_pdf(pdf_path)
print(text[:2000])  # preview

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experime

In [ ]:
def extract_abstract(text):
    # Try to find 'Abstract' section more robustly
    abstract_pattern = re.compile(
        r"(?:^|\n)(abstract|ABSTRACT)\s*[:\-]?\s*\n?(.*?)(?=\n\s*[A-Z][A-Z\s]{3,}\n)",  # Next section is ALL CAPS
        re.DOTALL
    )
    match = abstract_pattern.search(text)
    if match:
        return match.group(2).strip()
    
    # Fallback 1: Try simpler pattern
    fallback_match = re.search(r"(?:^|\n)(abstract|ABSTRACT)[\s:\-]*\n?(.*?)(\n\n|\n[A-Z]{2,})", text, re.DOTALL)
    if fallback_match:
        return fallback_match.group(2).strip()

    # Fallback 2: Return first 1000 characters if all else fails
    return text[:1000].strip()

abstract = extract_abstract(text)
print(abstract)

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experime

In [7]:
def simplify_abstract(abstract, model_path):
    prompt = f"""<|system|>
You simplify research abstracts into plain English.
<|user|>
Simplify this abstract:
\"\"\"{abstract.strip()}\"\"\"
<|assistant|>
"""

    with tempfile.NamedTemporaryFile(mode='w+', delete=False) as f:
        f.write(prompt)
        f.flush()

        result = subprocess.run(
            [
                "./llama.cpp/build/bin/llama-cli",
                "-m", model_path,
                "-f", f.name,
                "--n-predict", "512",
                "--temp", "0.7"
            ],
            capture_output=True,
            text=True
        )
        return result.stdout

In [8]:
def extract_last_assistant_response(output_text: str) -> str:
    parts = output_text.split("<|assistant|>")
    response = parts[-1] if len(parts) > 1 else output_text
    return response.replace("[end of text]", "").strip()

In [10]:
abstract = "The dominant sequence transduction models are based on complex recurrent " \
"or convolutional neural networks that include an encoder and a decoder. The best performing " \
"models also connect the encoder and decoder through an attention mechanism. We propose a " \
"new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing " \
"with recurrence and convolutions entirely. Experiments on two machine translation tasks show these " \
"models to be superior in quality while being more parallelizable and requiring significantly less " \
"time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, " \
"improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 " \
"English-to-French translation task, our model establishes a new single-model state-of-the-art " \
"BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training " \
"costs of the best models from the literature. We show that the Transformer generalizes well to " \
"other tasks by applying it successfully to English constituency parsing both with large and limited training data."

In [18]:
model_path = "models/mistral-7b-instruct-v0.1.Q4_K_M.gguf"

simplified_raw = simplify_abstract(abstract, model_path)
cleaned_output = extract_last_assistant_response(simplified_raw)

print("📝 Simplified Abstract:\n", cleaned_output)

# Optional: save output
# with open("outputs/simplified_output.txt", "w") as f:
#     f.write(cleaned_output)

📝 Simplified Abstract:
 The Transformer is a new network architecture that uses attention mechanisms to simplify complex recurrent or convolutional neural networks used for sequence transduction tasks like machine translation. It outperforms existing models in quality while being more parallelizable and requiring less time to train. The Transformer achieves a new state-of-the-art BLEU score of 41.8 on the English-to-French translation task and 28.4 BLEU on the English-to-German translation task, improving over previous best results. It also generalizes well to other tasks and can be applied successfully to English constituency parsing with limited training data.
